<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-el-csv-que-vamos-a-usar" data-toc-modified-id="Cargamos-el-csv-que-vamos-a-usar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cargamos el <code>csv</code> que vamos a usar</a></span></li><li><span><a href="#Definición-de-variables" data-toc-modified-id="Definición-de-variables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Definición de variables</a></span></li><li><span><a href="#Creamos-la-clase" data-toc-modified-id="Creamos-la-clase-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creamos la clase</a></span><ul class="toc-item"><li><span><a href="#Hacemos-las-llamadas-a-los-métodos-de-la-Clase" data-toc-modified-id="Hacemos-las-llamadas-a-los-métodos-de-la-Clase-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hacemos las llamadas a los métodos de la Clase</a></span><ul class="toc-item"><li><span><a href="#Iniciamos-la-Clase" data-toc-modified-id="Iniciamos-la-Clase-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Iniciamos la Clase</a></span></li><li><span><a href="#Llamada-a-la-API" data-toc-modified-id="Llamada-a-la-API-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Llamada a la API</a></span></li><li><span><a href="#Limpiamos-los-dataframes" data-toc-modified-id="Limpiamos-los-dataframes-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Limpiamos los dataframes</a></span></li><li><span><a href="#Juntamos-los-dataframes" data-toc-modified-id="Juntamos-los-dataframes-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Juntamos los dataframes</a></span></li><li><span><a href="#Chequeamos-como-están-los-datos-de-&quot;limpios&quot;" data-toc-modified-id="Chequeamos-como-están-los-datos-de-&quot;limpios&quot;-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Chequeamos como están los datos de "limpios"</a></span></li></ul></li></ul></li></ul></div>

En este jupyter vamos a hacer lo mismo que en el anterior, pero en este caso lo haremos definiendo una clase que nos permita hacer todo el proceso. 

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Cargamos el `csv` que vamos a usar

In [2]:
df = pd.read_csv("./datos_Madrid.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,...,longitud,ciudad,seeing_x,transparency_x,seeing_y,transparency_y,ciudad_x.1,seeing,transparency,ciudad_y.1
0,3,2,10,none,0,-1,73,clearnight,NW,5,...,-370.256,Madrid,NaN,NaN,2.0,2.0,NaN,2,2,Madrid
1,3,2,10,none,0,-1,73,clearnight,NW,5,...,-370.256,NaN,NaN,NaN,NaN,NaN,Madrid,2,2,Madrid


# Definición de variables 

Definimos las variables que vamos a necesitar para hacer las llamadas a cada uno de los métodos de la clase. 

In [3]:
latitud = 40.4165
longitud = -3.70256
producto = "civil"
producto2 = "astro"
ciudad = "Madrid"


# Creamos la clase 

In [4]:
class Extraccion: 
    
    def __init__(self, lat, lon, ciudad):
        self.lat = lat
        self.lon = lon
        self.ciudad = ciudad
        self.producto = producto
        
    def llamada_API(self, producto):
        
        self.producto = producto
    
        # hacemos la llamada  a la API
        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={producto}&output=json'

        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

        # convertimos los resultados en un dataframe: 
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        return df
    
    def limpiar_civil(self, df): 

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["ciudad"] = ciudad
        return df
    
    
    
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 
    
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_civil], axis = 0)

        # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
        # en este caso el how lo ponemos como left ya que queremos que se quede con toda la info que en la primera tabla que le pasamos que es la que tiene toda la información
        df_completo = pd.merge(df_completo , df_visibilidad , on=['fecha', "timepoint"], how = "inner")

        # guardamos los datos
        df_completo.to_pickle('./datos_actualizados.pkl')
        df_completo.to_csv('./datos_actualizados.csv')

        return df_completo
    

    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        # quitar columnas repetidas:
        
        df.drop(["ciudad_y"], axis = 1, inplace = True)
        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('datos/datos_Madrid.pkl')
        df.to_csv('datos/datos_Madrid.csv')

        return df

## Hacemos las llamadas a los métodos de la Clase 

### Iniciamos la Clase 

In [5]:
api = Extraccion(latitud, longitud, ciudad)

In [6]:
api

### Llamada a la API 

1️⃣ Llamamos al producto "civil"

In [7]:
df_civil = api.llamada_API("civil")
df_civil.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed
0,3,7,2,none,0,24,44%,mcloudyday,SE,2
1,6,8,-1,none,0,26,39%,cloudyday,SE,3


2️⃣ Llamamos al producto "astro"

In [8]:
df_astro = api.llamada_API("astro")
df_astro.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,seeing,transparency,lifted_index,rh2m,temp2m,prec_type,wind10m.direction,wind10m.speed
0,3,7,1,2,2,4,24,none,SE,2
1,6,8,1,2,-1,3,26,none,SE,3


### Limpiamos los dataframes

In [9]:
df_civil = api.limpiar_civil(df_civil)
df_civil.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,7,2,none,0,24,44%,mcloudyday,SE,2,2022-05-18,40.4165,-3.70256,Madrid
1,6,8,-1,none,0,26,39%,cloudyday,SE,3,2022-05-18,40.4165,-3.70256,Madrid


In [10]:
df_astro = api.limpiar_astro(df_astro)
df_astro.head(2)

/var/folders/rp/8_fhdgqx4mv2kkv4dswj4tq40000gn/T/ipykernel_4383/1991873112.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fecha"] = hoy
/var/folders/rp/8_fhdgqx4mv2kkv4dswj4tq40000gn/T/ipykernel_4383/1991873112.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ciudad"] = self.ciudad


,seeing,transparency,timepoint,fecha,ciudad
0,1,2,3,2022-05-18,Madrid
1,1,2,6,2022-05-18,Madrid


### Juntamos los dataframes 

In [11]:
df_completo = api.juntar_dfs(df, df_civil, df_astro)
df_completo.head(2)

/var/folders/rp/8_fhdgqx4mv2kkv4dswj4tq40000gn/T/ipykernel_4383/1991873112.py:74: FutureWarning: Passing 'suffixes' which cause duplicate columns {'transparency_x', 'seeing_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_completo = pd.merge(df_completo , df_visibilidad , on=['fecha', "timepoint"], how = "inner")


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,...,transparency_x,seeing_y,transparency_y,ciudad_x.1,seeing_x,transparency_x,ciudad_y.1,seeing_y,transparency_y,ciudad_y
0,3,7,2,none,0,24,44%,mcloudyday,SE,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Madrid
1,6,8,-1,none,0,26,39%,cloudyday,SE,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Madrid


In [12]:
df_completo.isnull().sum()

timepoint             0
cloudcover            0
lifted_index          0
prec_type             0
prec_amount           0
temp2m                0
rh2m                  0
weather               0
wind10m.direction     0
wind10m.speed         0
fecha                 0
latitud               0
longitud              0
ciudad_x              0
seeing_x             24
transparency_x       24
seeing_y             24
transparency_y       24
ciudad_x.1           24
seeing_x             24
transparency_x       24
ciudad_y.1           24
seeing_y              0
transparency_y        0
ciudad_y              0
dtype: int64

### Chequeamos como están los datos de "limpios"

In [13]:
api.chequear_datos(df_completo)

Las columnas son: 

['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount', 'temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed', 'fecha', 'latitud', 'longitud', 'ciudad_x', 'seeing_x', 'transparency_x', 'seeing_y', 'transparency_y', 'ciudad_x.1', 'seeing_x', 'transparency_x', 'ciudad_y.1', 'seeing_y', 'transparency_y', 'ciudad_y']
-----------------------------------------
Los tipos de datos que tenemos son: 

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                  object
weather               object
wind10m.direction     object
wind10m.speed          int64
fecha                 object
latitud              float64
longitud             float64
ciudad_x              object
seeing_x             float64
transparency_x       float64
seeing_y             float64
transparency_y       float64
ciudad_x.1            object
seeing

In [14]:
# definimos las columnas que tienen nulos

columnas_con_nulos = ["seeing", "transparency", "fecha", "latitud", "longitud", 'ciudad_x', "ciudad_y"]

In [ ]:
df_final = api.limpiar_dataframe(df_completo, columnas_con_nulos)
df_final.head(2)

Y listo!! Si cada día ejecutaramos este código tendríamos actualizado al día la información meteorológica para la ciudad de Madrid 💪🏽

**EJERCICIOS** 

En este ejercicio tendréis que usar la misma API que hemos usado hasta ahora y elegir otro punto de información (hasta ahora hemos usado `civil` y `astro`). Descarga los datos, limpialos y juntalos con los datos de los otros puntos de información que hemos obtenido durante esta lección. 

📌 **NOTA** Al juntar toda la información puede que haya algunas columnas que tengan la misma información. Debemos evitar información duplicada en nuestro *dataframe*, por lo que tendremos que hacer una buena selección de las columnas que queremos conservar.

💪🏽 **BONUS** Puede que algunas de las columnas que tengamos de la nueva llamada a la API tengan una pinta como esta 👇🏽

```python
[{'layer': '950mb', 'rh': 9},
 {'layer': '900mb', 'rh': 11},
 {'layer': '850mb', 'rh': 12},
 {'layer': '800mb', 'rh': 6},
 {'layer': '750mb', 'rh': 4},
 {'layer': '700mb', 'rh': 5},
 {'layer': '650mb', 'rh': 7},
 {'layer': '600mb', 'rh': 15},
 {'layer': '550mb', 'rh': 16},
 {'layer': '500mb', 'rh': 13},
 {'layer': '450mb', 'rh': 9},
 {'layer': '400mb', 'rh': 11},
 {'layer': '350mb', 'rh': 10},
 {'layer': '300mb', 'rh': 10},
 {'layer': '250mb', 'rh': 8},
 {'layer': '200mb', 'rh': -3}]
```

El objetivo de este bonus es serparar cada elemento de esa lista de diccionarios en columnas diferentes. 
